In [1]:
!pip install streamlit pyngrok


In [2]:
%%writefile sales_dashboard_streamlit_app.py

import streamlit as st
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from io import BytesIO

st.set_page_config(page_title="Financial Analysis Platform", layout="wide")

# Light Theme Styling
st.markdown("""
    <style>
    body, .reportview-container {
        background-color: #f9f9f9;
        color: #222;
        font-family: "Segoe UI", sans-serif;
    }
    .stButton > button {
        background-color: #1f77b4;
        color: white;
        padding: 0.5em 1em;
        border-radius: 8px;
        font-size: 16px;
    }
    .stDataFrame, .stTable {
        background-color: white;
        border-radius: 10px;
        padding: 10px;
    }
    </style>
""", unsafe_allow_html=True)

st.title("📊 Financial Analysis Platform")

uploaded_file = st.file_uploader("Upload your sales data CSV file", type=["csv"])

if uploaded_file:
    df = pd.read_csv(uploaded_file)
    df.columns = df.columns.str.strip()

    st.header("📂 Raw Data Preview")
    st.dataframe(df.head())

    df['Order Date'] = pd.to_datetime(df['Order Date'], errors='coerce')
    df.dropna(subset=['Order Date'], inplace=True)
    df['Month'] = df['Order Date'].dt.month
    df['Hour'] = df['Order Date'].dt.hour
    df['Year'] = df['Order Date'].dt.year

    st.markdown("---")
    st.header("📈 Visual Analysis")

    def plot_and_download(fig, filename, label):
        buf = BytesIO()
        fig.savefig(buf, format="png")
        st.pyplot(fig)
        st.download_button(label=f"Download {label}", data=buf.getvalue(), file_name=filename, mime="image/png")

    # Total Amount by Month
    st.subheader("📅 Total Amount by Month")
    monthly = df.groupby('Month')['Amount'].sum()
    fig, ax = plt.subplots()
    sns.barplot(x=monthly.index, y=monthly.values, palette="Blues_d", ax=ax)
    ax.set_title("Monthly Sales")
    plot_and_download(fig, "monthly_sales.png", "Monthly Sales Chart")

    # Total Amount by City
    st.subheader("🏙️ Total Amount by City")
    city = df.groupby('City')['Amount'].sum().sort_values(ascending=False)
    fig, ax = plt.subplots()
    sns.barplot(x=city.values, y=city.index, palette="coolwarm", ax=ax)
    ax.set_title("Sales by City")
    plot_and_download(fig, "city_sales.png", "City Sales Chart")

    # Top 10 Sub-Categories by Profit
    st.subheader("📦 Top 10 Sub-Categories by Profit")
    top_subcats = df.groupby('Sub-Category')['Profit'].sum().sort_values(ascending=False).head(10)
    fig, ax = plt.subplots()
    sns.barplot(x=top_subcats.values, y=top_subcats.index, palette="crest", ax=ax)
    ax.set_title("Top Sub-Categories by Profit")
    plot_and_download(fig, "top_subcategories_profit.png", "Top Sub-Categories Chart")

    # Monthly Profit Trend
    st.subheader("📉 Monthly Profit Trend")
    monthly_profit = df.groupby('Month')['Profit'].sum()
    fig, ax = plt.subplots()
    sns.lineplot(x=monthly_profit.index, y=monthly_profit.values, marker='o', color='purple', ax=ax)
    ax.set_title("Monthly Profit Trend")
    plot_and_download(fig, "monthly_profit_trend.png", "Monthly Profit Chart")

    # Orders by Category (Pie Chart)
    st.subheader("🍕 Orders by Category")
    category_counts = df['Category'].value_counts()
    fig, ax = plt.subplots()
    ax.pie(category_counts.values, labels=category_counts.index, autopct='%1.1f%%', startangle=140, colors=sns.color_palette('pastel'))
    ax.axis('equal')
    plot_and_download(fig, "orders_by_category.png", "Category Pie Chart")

    # Payment Mode Analysis
    if 'PaymentMode' in df.columns:
        st.subheader("💳 Payment Mode Distribution")
        payment_mode = df['PaymentMode'].value_counts()
        fig, ax = plt.subplots()
        ax.pie(payment_mode, labels=payment_mode.index, autopct='%1.1f%%', colors=sns.color_palette("Set2"))
        ax.axis('equal')
        plot_and_download(fig, "payment_mode_distribution.png", "Payment Mode Chart")

    # Option to download cleaned Data
    csv = df.to_csv(index=False).encode('utf-8')
    st.download_button("Download Cleaned CSV", csv, "cleaned_data.csv", "text/csv")

else:
    st.info("📁 Please upload a valid CSV file to begin analysis.")


Overwriting sales_dashboard_streamlit_app.py


In [3]:
!ngrok config add-authtoken 2xVCyikcFmKpFib3VF8cd9cTUw5_2XeRR8D9hTNEncBCwK2EE


Authtoken saved to configuration file: C:\Users\tripu\AppData\Local/ngrok/ngrok.yml


In [4]:
import os
import subprocess
import time
from pyngrok import ngrok

# Step 1: Kill any existing Streamlit and ngrok processes
print("🔄 Killing existing processes...")
os.system("taskkill /F /IM streamlit.exe >nul 2>&1")
os.system("taskkill /F /IM ngrok.exe >nul 2>&1")

# Step 2: Start Streamlit app
print("🚀 Starting Streamlit app...")
streamlit_process = subprocess.Popen(["streamlit", "run", "sales_dashboard_streamlit_app.py"],
                                     stdout=subprocess.DEVNULL,
                                     stderr=subprocess.DEVNULL)

# Optional: Wait to ensure the Streamlit app has started
time.sleep(5)

# Step 3: Start ngrok tunnel
print("🌐 Starting ngrok tunnel...")
public_url = ngrok.connect(8501)
print(f"✅ Streamlit app is live at: {public_url}")



🔄 Killing existing processes...
🚀 Starting Streamlit app...
🌐 Starting ngrok tunnel...
✅ Streamlit app is live at: NgrokTunnel: "https://84c1-2401-4900-60d3-47bd-c46d-fb60-a83f-c9f1.ngrok-free.app" -> "http://localhost:8501"
